# Wind turbine models

FLORIS generally represents wind turbines as actuator disks specified by a power curve and a
thrust coefficient curve (both specified as a function of wind speed). We can easily investigate the
power and thrust coefficients of a turbine by running a single-turbine `FlorisModel` using a range
of wind speeds.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from floris import FlorisModel, TimeSeries

n_wind_speeds = 100
wind_speeds = np.linspace(0.1, 30, n_wind_speeds)

fmodel = FlorisModel("defaults") # Defaults to NREL 5MW turbine
fmodel.set(
    wind_data=TimeSeries(
        wind_directions=np.zeros(n_wind_speeds),
        wind_speeds=wind_speeds,
        turbulence_intensities=0.06
    ),
    layout_x=[0],
    layout_y=[0],
    wind_shear=0.0
)

fmodel.run()

powers = fmodel.get_turbine_powers()
thrust_coefficients = fmodel.get_turbine_thrust_coefficients()

fig, ax = plt.subplots(2, 1, sharex=True)
ax[0].plot(wind_speeds, powers)
ax[0].grid()
ax[0].set_ylabel("Power [kW]")
ax[1].plot(wind_speeds, thrust_coefficients)
ax[1].grid()
ax[1].set_ylabel("Thrust coefficient [-]")
ax[1].set_xlabel("Wind speed [m/s]")
ax[1].set_xlim([0, 30])

## Prepackaged turbine models

FLORIS ships with three reference wind turbine models: the NREL 5MW turbine
{cite:t}`jonkman_NREL5MW_2009`,
the IEA 10 MW turbine {cite:t}`kainz_IEA10MW_2024`, and the IEA 15 MW turbine
{cite:t}`gaertner_IEA15MW_2020`. The
turbine models used for FLORIS simulations can be changed by specifying the `turbine_type` keyword
argument to `FlorisModel.set()`. See {ref}`turbine_library` for more details.

In [ ]:
fig, ax = plt.subplots(2, 1, sharex=True)

turbine_models = ["nrel_5MW", "iea_10MW", "iea_15MW"]

for turbine_model in turbine_models:
    fmodel.set(turbine_type=[turbine_model], reference_wind_height=-1)

    fmodel.run()

    powers = fmodel.get_turbine_powers()
    thrust_coefficients = fmodel.get_turbine_thrust_coefficients()

    ax[0].plot(wind_speeds, powers, label=turbine_model)
    ax[1].plot(wind_speeds, thrust_coefficients)


ax[0].grid()
ax[0].set_ylabel("Power [kW]")
ax[0].legend()
ax[1].grid()
ax[1].set_ylabel("Thrust coefficient [-]")
ax[1].set_xlabel("Wind speed [m/s]")
ax[1].set_xlim([0, 30])

## User-defined wind turbine models

Users may also provide their own wind turbine models, provided that they contain the appropriate
information. To include your own wind turbine model in your main FLORIS input YAML, use the
`!include` specifier, e.g.
```
  turbine_type:
  - !include path/to/your/turbine.yaml
```

You can also `set` the `turbine_type` to your own turbine using the path, e.g.
```python
fmodel.set(turbine_type=["path/to/your/turbine.yaml"])
```

The following pages describe in closer detail how wind turbines are implemented in FLORIS and
provide information on advanced wind turbine operation and modeling.

```{note}
The `TurbineInterface` and `TurbineLibrary` classes are now deprecated and will be removed in a
future release. Users should simply use an instantiated `FlorisModel` to investigate wind turbine
models.
```